# Convolutional Sentiment Classifier

In this notebook, we build a *convolutional* neural net to classify IMDB movie reviews by their sentiment.

### Load Dependencies

In [1]:
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D # new! 
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 

### Set Hyperparameters

In [2]:
# output directory name:
output_dir = 'model_output/conv'

# training :
epochs = 4
batch_size = 128

# vector_spece embedding:
n_dim = 64
n_unique_words = 5000
max_review_length = 400
pad_type = trunc_type = 'pre'
drop_embed = 0.2 # new!

# Convolutional Layer Architecture:
n_conv = 256 # filters, a.k.a kernels
k_conv = 3 # kernal length

# Dense Layer Architecture:
n_dense = 256
dropout = 0.2

### Load Data


In [3]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

### Preprocess Data

In [4]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)


### Design Neural Network Architecture

In [8]:
model = Sequential()

# first hidden layer:
model.add(Embedding(n_unique_words, n_dim, input_length = max_review_length))
model.add(SpatialDropout1D(drop_embed))

# second hidden layer:
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(GlobalMaxPooling1D())

# third hidden layer:
# model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(GlobalMaxPooling1D())

# fourth hidden layer:
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))

# output layer:
model.add(Dense(1, activation='sigmoid'))


In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 400, 64)           320000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 400, 64)          0         
 lDropout1D)                                                     
                                                                 
 conv1d_3 (Conv1D)           (None, 398, 256)          49408     
                                                                 
 conv1d_4 (Conv1D)           (None, 396, 256)          196864    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 256)              

### Configure Model

In [10]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [11]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train!

In [13]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

Epoch 1/4
 52/196 [======>.......................] - ETA: 14:12 - loss: 0.6865 - accuracy: 0.5299

KeyboardInterrupt: 

### Evaluate

In [ ]:
model.load_weights(output_dir+"/weights.03.hdf5")

In [ ]:
y_hat = model.predict(x_valid)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
"{:0.2f}".format(roc_auc_score(y_valid, y_hat)*100.0)